## codecs: кодирование/декодирование строк 

Модуль codecs представляет из себя потоковый и файловый интерфейсы для перекодировки текстовых данных между различными представлениями.

Чаще используется для работы с текстом в представлении Unicode.

In [47]:
import binascii

def to_hex(t,nbytes):
    chars_per_item = nbytes * 2
    hex_version = binascii.hexlify(t)
    return b' '.join(
        hex_version[start:start + chars_per_item]
        for start in range (0, len(hex_version), chars_per_item)
    )

if __name__ == '__main__':
    print(to_hex(b'abcdef', 1))
    print(to_hex(b'abcdef', 2))

b'61 62 63 64 65 66'
b'6162 6364 6566'


Модуль binascii использован в функции для получения шестнадцатиричного представления входной байтовой строки, далее- выставляет пробелы между каждыми nbytes байтами, прежде чем вернуть значение. 


Модуль codecs предоставляет классы, управляющие кодированием и декодированием данных, избавляя приложение от необходимости выполнять работу 

In [70]:
import unicodedata 
import codecs


text = 'franpais' 

print ('Raw : {!r}'.format(text)) #представление класса unicode 
for c in text:
    print ('{!r}: {}'. format(c, unicodedata.name(c, c)))
print ('UTF-8: {!r}'.format(to_hex(text.encode('utf-8'), 1)))
print ('UTF-16: {!r}'.format(to_hex(text.encode('utf-16'), 2)))
#кодировка c использованием схем UTF-8 и UTF-16 


Raw : 'franpais'
'f': LATIN SMALL LETTER F
'r': LATIN SMALL LETTER R
'a': LATIN SMALL LETTER A
'n': LATIN SMALL LETTER N
'p': LATIN SMALL LETTER P
'a': LATIN SMALL LETTER A
'i': LATIN SMALL LETTER I
's': LATIN SMALL LETTER S
UTF-8: b'66 72 61 6e 70 61 69 73'
UTF-16: b'fffe 6600 7200 6100 6e00 7000 6100 6900 7300'


Результат вывода: байты в виде шестнадцатеричных значениях 

Результатом кодирования строки str является bytes


Если последовательность кодированных байтов задана в виде экземпляра
bytes, метод decode () преобразует байты в кодовые точки и возвращает последовательность в виде экземпляра str

(!) Вводимый тип не зависит от выбора исходной кодировки

In [71]:

text = 'franpais' 
encoded = text.encode ('utf-8')
decoded = encoded.decode ('utf-8')

print('Original :', repr(text))
print('Encoded :', to_hex(encoded, 1), type(encoded))
print('Decoded :', repr(decoded), type(decoded))

Original : 'franpais'
Encoded : b'66 72 61 6e 70 61 69 73' <class 'bytes'>
Decoded : 'franpais' <class 'str'>


#### Порядок байтов


В модуле codecs определены константы, которые служат маркерами порядка байтов, используемыми кодировками UTF-16 и UTF-32

Например, кодировка UTF-16 определена таким образом, что кодам 0xFFFE и 0xFEFF не соответствуют никакие допустимые символы, и их можно использовать для указания порядка байтов.

In [87]:
import codecs

BOM_TYPES = [
    'BOM', 'BOM_BE', 'BOM_LE',
    'BOM_UTF8',
    'BOM_UTF16', 'BOM_UTF16_BE', 'BOM_UTF16_LE',
    'BOM_UTF32', 'BOM_UTF32_BE', 'BOM_UTF32_LE',
]

for name in BOM_TYPES:
    print('{:12} : {}'.format(
    name, to_hex(getattr(codecs, name), 2)))

BOM          : b'fffe'
BOM_BE       : b'feff'
BOM_LE       : b'fffe'
BOM_UTF8     : b'efbb bf'
BOM_UTF16    : b'fffe'
BOM_UTF16_BE : b'feff'
BOM_UTF16_LE : b'fffe'
BOM_UTF32    : b'fffe 0000'
BOM_UTF32_BE : b'0000 feff'
BOM_UTF32_LE : b'fffe 0000'


Для констант BOM, BOM_UTF16 и BOM_UTF32 автоматически устанавливаются
значения, соответствующие прямому (“от старшего к младшему” — big-endian) или обратному (“от младшего к старшему“ — little-endian) порядку байтов, в зависимости от собственного машинного порядка следования байтов в данной системе

Порядок следования байтов автоматически распознается и обрабатывается де-
кодерами модуля codecs,

#### Иные кодировки  

Несмотря на то что в большинстве предыдущих примеров использовались кодировки Unicode, модуль codecs может работать не только c ними. Например,
Python включает кодеки для работы c кодировками base64, bzip2, ROT-13, ZIP и многими другими.

In [91]:
import codecs
import io

buffer = io.StringIO()
stream = codecs.getwriter('rot_13')(buffer)

text = 'abcdefghijklmnopqrstuvwxyz'

stream.write(text)
stream.flush()

print('Original:', text)
print('ROT-13 :', buffer.getvalue())

Original: abcdefghijklmnopqrstuvwxyz
ROT-13 : nopqrstuvwxyzabcdefghijklm


Использование модуля codecs для обертывания потока данных предоставля-
ет более простой интерфейс, чем работа непосредственно c модулем zlib

In [95]:

buffer = io.BytesIO()
stream = codecs.getwriter('zlib')(buffer)

text = b'abcdefghijklmnopqrstuvwxyz\n' * 50

stream.write(text)
stream.flush()

print('Original length :', len(text))
compressed_data = buffer.getvalue()
print('ZIP compressed :', len(compressed_data))

buffer = io.BytesIO(compressed_data)
stream = codecs.getreader('zlib')(buffer)

first_line = stream.readline()
print('Read first line :', repr(first_line))

uncompressed_data = first_line + stream.read()
print('Uncompressed :', len(uncompressed_data))
print('Same :', text == uncompressed_data)

Original length : 1350
ZIP compressed : 48
Read first line : b'abcdefghijklmnopqrstuvwxyz\n'
Uncompressed : 1350
Same : True


Не все алгоритмы сжатия или кодирования поддерживают чтение данных порциями через потоковый интерфейс c помощью функций readline () или read (),
поскольку они должны находить конец сжатого сегмента для распаковки данных.

Если программа не может сохранить полный распакованный набор данных в памяти, то вместо модуля codecs следует использовать возможности инкрементного доступа библиотеки сжатия.

#### Unicode и сетевой обмен данными


Сетевые сокеты — это байтовые потоки, и в отличие от стандартных потоков
вода-вывода они по умолчанию не поддерживают преобразование данных.

In [ ]:
#cервер пытается возвращать переданные ему данные отправителю
import sys
import socketserver

class Echo(socketserver.BaseRequestHandler):
    
    def handle(self):
        # Получить байты и вернуть их клиенту
        data = self.request.recv(1024)
        self.request.send(data)
        return
    
if __name__ == '__main__' :
    import codecs
    import socket
    import threading

    address = ('localhost', 0) # позволить ядру назначить порт
    server = socketserver.TCPServer(address, Echo)
    ip, port = server.server_address # Какой порт назначен?

    t = threading.Thread(target=server.serve_forever)
    t.setDaemon(True) # превратить поток в поток-демон
    t.start()

    # Подключение к серверу
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((ip, port))
    
    # Отправка данных
    text = 'frangais'
    print('Sending :', repr(text))
    outgoing.write(text)
    outgoing.flush()
    
    # Получение ответа
    response = incoming.read()
    print('Received:', repr(response))
    
    # Освобождение ресурсов
    s.close()
    server.socket.close()
"""Данные можно было преобразовывать в байты перед каждым вызовом метода
send (), но поскольку этого не было сделано, попытка вызова send () приводит к
ошибке."""


#### Домашнее задание 

Преобразовать словосочетание "hello world" в численное значение при помощи функции crc32 модуля binascii